In [3]:
import pandas as pd
import geopandas as gpd
import numpy as np
import time
from helper_functions import *
from all_config import africa_countries, latam_countries, asia_countries, europe_countries


Countries


In [4]:
# update this to have validation test functions for
# each single map,
# ggit map
# multi tracker map
# gas only multi tracker map
# for europe remember ggit hy is the one not ggit

# Then run for all remaining maps needed: AET (multi), LATAM (multi), agt (gas only), egt (gas only), ggit (special), gmet (single)
# Verify that new admin panel aligns with current code config for geo and fuel filters
# Use date last researched and file title to verify version

<!-- Map: asia
Total number of DataFrames: 2
Tracker counts: Counter({'GOGPT': 4858, 'GGIT': 1645, 'GOGET': 907, 'GGIT-lng': 481})
----------------------------------------
Map: europe
Total number of DataFrames: 2
Tracker counts: Counter({'GOGPT': 2675, 'GOGET': 1202, 'GGIT': 1062, 'GGIT-lng': 168}) -->

In [5]:


# NOTE: CONFIGURATIONS TO BE CHANGED AS NEEDED PER TESTER AND CYCLE
map_to_test = 'africa' 

testing_source_path = '/Users/gem-tah/GEM_INFO/GEM_WORK/earthrise-maps/testing/source/'
testing_final_path = '/Users/gem-tah/GEM_INFO/GEM_WORK/earthrise-maps/testing/final/'
testing_errors_path = '/Users/gem-tah/GEM_INFO/GEM_WORK/earthrise-maps/testing/'


# use the same version of each tracker as was used for current version of Africa Energy map
coal_mines_file = 'Global-Coal-Mine-Tracker-April-2024 DATA TEAM COPY.xlsx' #'Global-Coal-Mine-Tracker-April-2024.xlsx'
gogpt_file = 'Global Oil and Gas Plant Tracker (GOGPT) - August 2024 DRAFT v2 DATA TEAM COPY.xlsx' #'Global Oil and Gas Plant Tracker (GOGPT) - February 2024 v4.xlsx'
gcpt_file = 'Global-Coal-Plant-Tracker-July-2024 DATA TEAM COPY.xlsx' #'Global-Coal-Plant-Tracker-July-2024.xlsx'
hydro_file = 'Global-Hydropower-Tracker-April-2024 DATA TEAM COPY.xlsx' #'Global-Hydropower-Tracker-April-2024.xlsx'
bio_file = 'Global-Bioenergy-Power-Tracker-GBPT-V2 DATA TEAM COPY.xlsx' #'Global-Bioenergy-Power-Tracker-2023-11.xlsx'
wind_file = 'Global-Wind-Power-Tracker-June-2024 DATA TEAM COPY.xlsx' #'Global-Wind-Power-Tracker-June-2024.xlsx'
goget_file = 'Global Oil and Gas Extraction Tracker - 2024-03-08_1205 DATA TEAM COPY.xlsx'
goget_formatted ='GOGET Earthgenome file 2024-07-03_200828.xlsx'
ggit_file = 'GEM-GGIT-Gas-Pipelines-2023-12 copy.geojson'
ggit_lng_file = 'GEM-GGIT-LNG-Terminals-2024-09 DATA TEAM COPY.geojson'
ggit_hy_file = 'Hydrogen Pipelines for EGT briefing in Feb 2024.xlsx'
nuclear_file = 'Global-Nuclear-Power-Tracker-July-2024 DATA TEAM COPY.xlsx'
coal_terminals_file = 'Global-Coal-Terminals-Tracker-December-2023 DATA TEAM COPY.xlsx'
goit_file = 'GEM-GOIT-Oil-NGL-Pipelines-2024-06 copy.geojson'
geothermal_file = 'Geothermal-Power-Tracker-May-2024 DATA TEAM COPY.xlsx'
solar_file = 'Global-Solar-Power-Tracker-June-2024 DATA TEAM COPY.xlsx'
coal_term_file = "Global-Coal-Terminals-Tracker-December-2023 DATA TEAM COPY.xlsx"

# create all source dfs
dfs = []
for sheet_name  in ['Main data', 'Production and reserves']: # TODO add in Scotts code, main data is main data, pull in prod as needed or use formatted for immediate test
    df = pd.read_excel(testing_source_path + goget_file, sheet_name=sheet_name,
        dtype=dtype_spec)
    dfs += [df]
goget = pd.concat(dfs).reset_index(drop=True)

goget_formatted = pd.read_excel(testing_source_path + goget_formatted, sheet_name='Data')
ggit_lng = gpd.read_file(testing_source_path + ggit_lng_file)
# ggit_hy = pd.read_excel(testing_source_path + ggit_hy_file, sheet_name = 'Gas Pipelines')
ggit = gpd.read_file(testing_source_path + ggit_file)
goit = gpd.read_file(testing_source_path + goit_file)
nuclear = pd.read_excel(testing_source_path + nuclear_file, sheet_name='Data',
        dtype=dtype_spec)
coal_term = pd.read_excel(testing_source_path + coal_terminals_file, sheet_name='Coal Terminals',
        dtype=dtype_spec)

dfs= []
for sheet_name in ['Data', 'Below Threshold']:
    df = pd.read_excel(
    testing_source_path + geothermal_file,
    sheet_name = sheet_name,
    dtype=dtype_spec
)
    dfs += [df]
geo = pd.concat(dfs).reset_index(drop=True)


dfs = []
for sheet_name in['20 MW+', '1-20 MW']:
    df = pd.read_excel(testing_source_path + solar_file, sheet_name=sheet_name,
        dtype=dtype_spec)
    dfs += [df]
solar = pd.concat(dfs).reset_index(drop=True) 

dfs = []
for sheet_name in ['Gas & Oil units', 'Sub-threshold units']:
    df = pd.read_excel(
        testing_source_path + gogpt_file, 
        sheet_name = sheet_name,
        dtype=dtype_spec)
    dfs += [df]
gogpt = pd.concat(dfs).reset_index(drop=True) 

gcpt = pd.read_excel(
    testing_source_path + gcpt_file,
    sheet_name = 'Units',
        dtype=dtype_spec
)
dfs = [] # initialize
for sheet_name in ['Global Coal Mine Tracker (Non-C', 'Global Coal Mine Tracker (Close']:
    df = pd.read_excel(
        testing_source_path +
        coal_mines_file,
        sheet_name = sheet_name,
        dtype=dtype_spec
        
    )
    if sheet_name == 'Global Coal Mine Tracker (Close':
        if 'Status' not in df.columns.to_list():
            df['Status'] = 'Retired'
    dfs += [df]
coal_mines = pd.concat(dfs).reset_index(drop=True)

hydro = pd.read_excel(
    testing_source_path + hydro_file,
    sheet_name = 'Data',
        dtype=dtype_spec
)

dfs= []
for sheet_name in ['Data', 'Below Threshold']:
    df = pd.read_excel(
    testing_source_path + wind_file,
    sheet_name = sheet_name,
    dtype=dtype_spec
)
    dfs += [df]
wind = pd.concat(dfs).reset_index(drop=True) 

dfs= []
for sheet_name in ['Data', 'Below Threshold']:
    df = pd.read_excel(
    testing_source_path + bio_file,
    sheet_name = sheet_name,
    dtype=dtype_spec
)
    dfs += [df]
bio = pd.concat(dfs).reset_index(drop=True)


coal_terminals = pd.read_excel(testing_source_path + coal_term_file,
                               sheet_name = 'Coal Terminals',
                               dtype=dtype_spec)




In [6]:


for file in os.listdir(testing_final_path): 
    if (map_to_test in file) & file.endswith(".geojson"):
        final_map = gpd.read_file(testing_final_path + file)
        print(file)

    elif (map_to_test in file) & file.endswith(".xlsx") & (new_release_date in file) & ('about' in file):
        # final_data = pd.ExcelFile(testing_final_path + file)
        # Read all sheets into a dictionary
        final_data_dict_with_about = pd.read_excel(testing_final_path + file, sheet_name=None)
        print(file)
 
    elif (map_to_test in file) & file.endswith(".xlsx") & (new_release_date in file):
        # final_data = pd.ExcelFile(testing_final_path + file)
        # Read all sheets into a dictionary
        final_data_dict = pd.read_excel(testing_final_path + file, sheet_name=None) 
        print(file)

    # elif ('Integrated' in file) & (file.endswith(".xlsx")):
    #     gipt_james_version = pd.read_excel(testing_final_path + file, sheet_name='Power facilities')
    #     print(file)
    # TODO add in the summary data files when we generate them
print(f"We will be testing for map {map_to_test}.")
print(f"Make sure all local source files have been updated for this cycle.")


africa-data-download August 2024.xlsx
africa_2024-10-22.geojson
We will be testing for map africa.
Make sure all local source files have been updated for this cycle.


In [7]:
final_map.columns

Index(['id', 'areas', 'url', 'name', 'unit-name', 'status', 'owner', 'parent',
       'start-year', 'capacity', 'subnat', 'region', 'tracker-acro',
       'official-name', 'tracker-custom', 'WKTFormat', 'Latitude', 'Longitude',
       'prod-coal', 'fuel', 'prod-oil', 'prod-year-oil', 'prod-gas',
       'prod-year-gas', 'river', 'scaling-capacity', 'capacity-table',
       'status-legend', 'areas-subnat-sat-display', 'tracker-display',
       'geometry'],
      dtype='object')

CHECK HERE

In [11]:
## For all multi maps not in gas only 
## So LATAM and Africa


if map_to_test not in gas_only_maps:
    # check goit, ggit, ggitlng, goget
    # discrepancy = [(goit,'GOIT'), (ggit, 'GGIT'), (goget,'GOGET'), (ggit_lng,'GGIT-lng')]
    list_of_trackers = [(coal_terminals, 'GCTT'), (coal_mines, 'GCMT'), (gcpt,'GCPT'), (gogpt, 'GOGPT'), (hydro, 'GHPT'), (wind,'GWPT'), (bio,'GBPT'),
                        (solar, 'GSPT'), (geo, 'GGPT'), (nuclear, 'GNPT'), (goget_formatted, 'GOGET'), (ggit_lng, 'GGIT-lng'), (ggit, 'GGIT'), (goit, 'GOIT')]

    # this one tests against actual data release for goget not the formatted map file
    list_of_trackers_dd = [(coal_terminals, 'GCTT Coal Terminals'), (coal_mines, 'GCMT Coal Mines'), (gcpt,'GCPT Coal Plants'), (gogpt, 'GOGPT Oil & GasPlants'), (hydro, 'GHPT Hydropower'), (wind,'GWPT Wind'), (bio,'GBPT Bioenergy'),
                        (solar, 'GSPT Solar'), (geo, 'GGPT Geothermal'), (nuclear, 'GNPT Nuclear'), (goget, 'GOGET Oil & Gas Extraction'), (ggit_lng, 'GGIT-lng LNG Terminals'), 
                        (ggit, 'GGIT Gas Pipelines'), (goit, 'GOIT Oil Pipelines')]

    # file_name = f'/Users/gem-tah/GEM_INFO/GEM_WORK/earthrise-maps/testing/output_africa_discrepancies_{new_release_date}.xlsx'
    # print(file_name)
    # with pd.ExcelWriter(file_name) as writer:
    if map_to_test == 'africa':
        needed_geo = africa_countries
        
    elif map_to_test == 'asia':
        needed_geo = asia_countries
        
    elif map_to_test == 'latam':
        needed_geo = latam_countries
        
    elif map_to_test == 'europe':
        needed_geo = europe_countries

    discrepancies = {}
    for source_tracker_tup in list_of_trackers:

        print("-" * 40)
        print(source_tracker_tup[1])
        tracker = source_tracker_tup[1]
        df = source_tracker_tup[0]

        renaming_sel = renaming_cols_dict[tracker]
        df = df.rename(columns=renaming_sel) 
        map_df = final_map[final_map['tracker-acro']==tracker]
        if tracker == 'GOGET':
            df = df[df['areas'].isin(needed_geo)] # TODO fix for this because it excludes when there is multiple countries and shouldn't OG0013150 for africa
            print(f'Source Data {tracker}: {len(df)}')
            print(f'Map Data {tracker}: {len(map_df)}')
            df_missing = df[~df['id'].isin(map_df['id'])]
            df_missing_reverse = map_df[~map_df['id'].isin(df['id'])]
            
            print('Map Discrepancy Count')
            print(len(df_missing))
            print(df_missing_reverse)
            if len(df_missing) > 0:
                # print(df_missing)
                discrepancies[tracker] = df_missing
                
        elif tracker == 'GCMT':
            df = df[df['areas'].isin(needed_geo)] # TODO fix for this because it excludes when there is multiple countries and shouldn't OG0013150 for africa
            print(f'Source Data {tracker}: {len(df)}')
            print(f'Map Data {tracker}: {len(map_df)}')
            df_missing = df[~df['id'].isin(map_df['id'])]
            df_missing_reverse = map_df[~map_df['id'].isin(df['id'])]
            
            print('Map Discrepancy Count')
            print(len(df_missing))
            print(df_missing_reverse)
            if len(df_missing) > 0:
                print(df_missing) 
            #     print(f'This is region for missing: {df_missing["region"]}')
            
            # # filter for no status
            # print('mapdf with no status gcmt:')
            # map_df['status'] = map_df['status'].fillna('')
            # map_df_no = map_df[map_df['status']=='']
            # print(map_df_no)
            # print('source df with no status gcmt:')   
            # df['status'] = df['status'].fillna('')
            # df_no = df[df['status']==''] 
            # print(df_no)   
        else:
            df = df[df['region']==f'{map_to_test.title()}']
            print(f'Source Data {tracker}: {len(df)}')
            print(f'Map Data {tracker}: {len(map_df)}')
            if 'id' not in df.columns:
                print(df.columns)
            df_missing = df[~df['id'].isin(map_df['id'])]
            df_missing_reverse = map_df[~map_df['id'].isin(df['id'])]

            if len(df_missing) > 0:
                print(f'df missing {df_missing}')
                if 'geometry' in df_missing.columns:
                    df_investigation_7 = df_missing[df_missing['geometry']!= '']
                    # print(df_investigation_7['geometry'])
                discrepancies[tracker] = df_missing
            print('Map Discrepancy Count')
            print(len(df_missing))
            # if len(df_missing_reverse) >0:
                # print(f'reverse {df_missing_reverse}')

            
    for source_tracker_tup in list_of_trackers_dd:

        print("-" * 40)
        tracker_acro = source_tracker_tup[1].split(' ')[0]
        tracker_official = source_tracker_tup[1].split(' ')[1:]
        print(tracker_acro)
        print(tracker_official)

        df = source_tracker_tup[0]

        renaming_sel = renaming_cols_dict[tracker_acro]
        print(renaming_sel)
        print(type(renaming_sel)) # dict but getting error of list
        df = df.rename(columns=renaming_sel) 
        dd_df = final_data_dict[tracker_official].rename(columns=renaming_sel) 

        if tracker_acro == 'GOGET':
            df = df[df['areas'].isin(needed_geo)]
            df_missing_dd = df[~df['id'].isin(dd_df['id'])]
            df_missing_dd_reverse = dd_df[~dd_df['id'].isin(df['id'])]

            print(f'Source Data {tracker_acro}: {len(df)}')
            print(f'Data Download Data {tracker_acro}: {len(dd_df)}')
            print('DD Discrepancy Count')
            print(len(df_missing_dd))
            if len(df_missing_dd) > 0:
                # print(df_missing_dd['region'])
                discrepancies[tracker_acro] = df_missing_dd

        else:
            df = df[df['region']=={map_to_test.title}]
            print(f'Source Data {tracker_acro}: {len(df)}')
            print(f'Data Download Data {tracker_acro}: {len(dd_df)}')
            df_missing_dd = df[~df['id'].isin(dd_df['id'])]
            df_missing_dd_reverse = dd_df[~dd_df['id'].isin(df['id'])]

            print('DD Discrepancy Count')
            print(len(df_missing_dd))
            if len(df_missing_dd) > 0:
                # print(df_missing_dd['region'])
                discrepancies[tracker_acro] = df_missing_dd
            # print(df_missing_dd_reverse)



----------------------------------------
GCTT
Source Data GCTT: 23
Map Data GCTT: 23
Map Discrepancy Count
0
----------------------------------------
GCMT
Source Data GCMT: 203
Map Data GCMT: 203
Map Discrepancy Count
0
Empty GeoDataFrame
Columns: [id, areas, url, name, unit-name, status, owner, parent, start-year, capacity, subnat, region, tracker-acro, official-name, tracker-custom, WKTFormat, Latitude, Longitude, prod-coal, fuel, prod-oil, prod-year-oil, prod-gas, prod-year-gas, river, scaling-capacity, capacity-table, status-legend, areas-subnat-sat-display, tracker-display, geometry]
Index: []

[0 rows x 31 columns]
----------------------------------------
GCPT
Source Data GCPT: 433
Map Data GCPT: 433
Map Discrepancy Count
0
----------------------------------------
GOGPT
Source Data GOGPT: 1090
Map Data GOGPT: 1090
Map Discrepancy Count
0
----------------------------------------
GHPT
Source Data GHPT: 219
Map Data GHPT: 219
Map Discrepancy Count
0
---------------------------------

KeyError: 'id'